In [1]:
!pip install pandas
!pip install pytrec_eval trectools
!pip install --upgrade transformers
!pip install --upgrade sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 524.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.0
    Uninstalling transformers-4.48.0:
      Successfully uninstalled transformers-4.48.0


In [2]:
import pandas as pd

column_names = ["query_id", "Q0", "doc_id", "rank", "score", "run_name"]

splade_df = pd.read_csv("splade-dev.trec", sep="\t", names=column_names)
bm25_df = pd.read_csv("bm25-t5-dev.trec", sep="\t", names=column_names)

In [3]:
splade_df.head()

,query_id,Q0,doc_id,rank,score,run_name
0,1048585,Q0,7187155,0,104472,R0
1,1048585,Q0,7187160,1,100811,R0
2,1048585,Q0,7187157,2,99206,R0
3,1048585,Q0,7187158,3,98698,R0
4,1048585,Q0,3100835,4,86255,R0


In [4]:
bm25_df.head()

,query_id,Q0,doc_id,rank,score,run_name
0,1048585,Q0,7187157,0,46.6189,R0
1,1048585,Q0,7187156,1,45.7317,R0
2,1048585,Q0,7187158,2,44.6009,R0
3,1048585,Q0,7617404,3,44.5052,R0
4,1048585,Q0,7187155,4,43.8783,R0


In [5]:
column_names_qrels = ["query_id", "zero", "doc_id", "relevance_score"]

qrels_df = pd.read_csv("qrels.dev.tsv", sep="\t", header=None, names=column_names_qrels)

qrels_dict = {}
for _, row in qrels_df.iterrows():
    query_id = row["query_id"]
    zero = row["zero"]
    doc_id = row["doc_id"]
    relevance_score = row["relevance_score"]

    if query_id not in qrels_dict:
        qrels_dict[query_id] = {}

    qrels_dict[query_id][doc_id] = relevance_score
    
qrels_df.head()

,query_id,zero,doc_id,relevance_score
0,1102432,0,2026790,1
1,1102431,0,7066866,1
2,1102431,0,7066867,1
3,1090282,0,7066900,1
4,39449,0,7066905,1


In [6]:
column_names_queries = ["query_id", "query"]

queries_df = pd.read_csv("queries.dev.tsv", sep="\t", header=None, names=column_names_queries)
queries_df.head()

,query_id,query
0,1048578,cost of endless pools/swim spa
1,1048579,what is pcnt
2,1048580,what is pcb waste
3,1048581,what is pbis?
4,1048582,what is paysky


In [7]:
queries = []

for query in queries_df["query"]:
    queries.append(query)

In [8]:
#from transformers import CompileConfig

from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from transformers import GenerationMixin
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
query_embeddings = model.encode(queries, convert_to_tensor=True)

# util.pytorch_cos_sim(sent_embedding1, sent_embedding2)

KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("naver/splade-cocondenser-ensembledistil")
model = AutoModelForMaskedLM.from_pretrained("naver/splade-cocondenser-ensembledistil")  

query_embeddings = model.encode(queries, convert_to_tensor=True)

query1 = query_embeddings[0]
query2 = query_embeddings[1]

sim = util.pytorch_cos_sim(query1, query2)

print(sim)